# LangChain Basic

In [1]:
!pip install -r ./requirements.txt -q

In [1]:
!pip show langchain

Name: langchain
Version: 0.0.335
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/richard/opt/anaconda3/lib/python3.9/site-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
!pip install langchain --upgrade -q

### Python-dotenv

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
os.environ.get('OPENAI_API_KEY')
os.environ.get('PINECONE_API_KEY')
os.environ.get('PINECONE_ENV')

'gcp-starter'

In [3]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 512}


In [4]:
output = llm('explain computer engineering in one sentence')
print(output)



Computer engineering is the design, development, and maintenance of computer hardware, software, and networks.


In [5]:
print(llm.get_num_tokens('explain computer engineering in one sentence'))

7


In [6]:
output = llm.generate(['... is the capital of England', 'What is the largest animal in the world?'])

In [7]:
print(output.generations)

[[Generation(text='?\n\nLondon.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe largest animal in the world is the Blue Whale. It can reach lengths of up to 100 feet and weigh up to 200 tons.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [8]:
print(output.generations[1][0].text)



The largest animal in the world is the Blue Whale. It can reach lengths of up to 100 feet and weigh up to 200 tons.


In [9]:
len(output.generations)

2

In [10]:
output = llm.generate(['Write an original tagline for sushi restaurant'] * 3)

In [11]:
print(output)

generations=[[Generation(text='\n\n"Sushi So Good It\'s Almost Too Good To Be True!"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Savor the flavor of fresh sushi!"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Satisfying sushi and more - Experience the Taste of Japan!"', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'completion_tokens': 43, 'total_tokens': 67, 'prompt_tokens': 24}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('af8a4425-b9f4-47eb-90ae-86116bfc1544')), RunInfo(run_id=UUID('935817fd-6aeb-43d7-8b77-cbe888195ba2')), RunInfo(run_id=UUID('e36f1e34-e478-47c3-8c92-90daff6d675b'))]


In [12]:
for output in output.generations:
    print(output[0].text, end='')



"Sushi So Good It's Almost Too Good To Be True!"

"Savor the flavor of fresh sushi!"

"Satisfying sushi and more - Experience the Taste of Japan!"

### ChatModels: GPT-3.5-Turbo and GPT-4

In [13]:
from langchain.schema import(AIMessage, HumanMessage, SystemMessage)
from langchain.chat_models import ChatOpenAI

In [14]:
chat = ChatOpenAI(model_name='gpt-4', temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content='You are a physicist.'),
    HumanMessage(content='explain quantum mechanics in one sentence') 
]
output = chat(messages)

In [15]:
print(output.content)

Quantum mechanics is a branch of physics that deals with phenomena on a very small scale, such as molecules, atoms, and subatomic particles, where particles can exist in multiple states at once and change state due to observation, described through probabilities rather than definite outcomes.


### Prompt Templates

In [16]:
from langchain import PromptTemplate

In [17]:
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)
print(prompt)

input_variables=['language', 'virus'] template='You are an experienced virologist.\nWrite a few sentences about the following {virus} in {language}.'


In [18]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7)
output = llm(prompt.format(virus='HIV', language='hindi'))
print(output)



एचआईवी एक ऐसा वायरस है जो हामी इंसुलिन सिस्टम (एचआईवी आईडी) को हैंडल करता है। यह अपने आप में बहुत तेजी से फैलता है और व्यक्ति को एचआईवी से जुड़े रोगों को संभावित कर


### Simple Chains

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-4', temperature=0.5)

template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)

output = chain.run({'virus': 'HSV', 'language': 'french'})

In [21]:
print(output)

L'HSV, ou virus de l'herpès simplex, est un virus qui provoque des infections courantes chez l'homme. Il existe deux types de ce virus : HSV-1, qui cause principalement l'herpès oral, et HSV-2, qui est principalement responsable de l'herpès génital. Les symptômes peuvent inclure des douleurs, des démangeaisons et des plaies dans les zones touchées.


### Sequential Chains

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''You are an experienced scientist and and Python programmer.
    Write a function that implements the concept of {concept}.'''
)

chain1 = LLMChain(llm=llm1, prompt=prompt1)

llm2 = ChatOpenAI(model_name='gpt-4', temperature=0.7, max_tokens=1024)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='''Given the Python function {function}, describe it as detailed as possible.'''
)

chain2 = LLMChain(llm=llm2, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.run('decision tree')



> Entering new SimpleSequentialChain chain...


def decision_tree(data, outcome):
    """
    This function uses a decision tree approach to classify a given dataset based on a given outcome.
    :param data: The dataset to be classified
    :param outcome: The outcome to be used for classification
    :return: A dictionary of the decision tree
    """
    # Find out the unique values in the outcome column
    unique_values = set(data[outcome])
    
    # Initialize a dictionary to store the decision tree
    decision_tree = {}
    
    # Iterate through the unique outcome values
    for value in unique_values:
        # Create a sub-dataset for each outcome value
        sub_data = data[data[outcome] == value]
        # Calculate the entropy of the sub-dataset
        entropy_sub_data = entropy(sub_data, outcome)
        # Find the best feature to split the dataset on
        best_feature = find_best_feature(sub_data, outcome)
        # Split the dataset on the best feature
        